In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Let's try to do our best with a few ML model

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [3]:
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)

from sklearn.cross_validation import KFold

C:\Users\Delanoue\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning:

numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.

C:\Users\Delanoue\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



## Data Preparation

We use the same data preparation we performed in the notebook called " A bad simple Linear Regression"


In a nutshell :

-I replace Nan values with the means from the train table

-I creat a One-Hot encoding column but it's useless here ...


In [4]:
from transplant.tools.dataset import Dataset 

dataset = Dataset()

train, test = dataset.get_static()

train1=train.apply(pd.to_numeric,errors='coerce').dropna(1, how="all")

mean_train1 = train1.mean()

train1=train1.fillna(mean_train1)

test1=test.apply(pd.to_numeric,errors='coerce').dropna(1, how="all").fillna(mean_train1)

drop_test1=[]
drop_train1=[]

for i in train1.columns :
    if not(i in test1.columns) :
        drop_train1+=[i]
        
for i in test1.columns :
    if not(i in train1.columns) :
        drop_test1+=[i]
drop_test1 , drop_train1

test1=test1.drop(drop_test1, axis=1)
train1=train1.drop(drop_train1, axis=1)

train1.shape, test1.shape

train2=train1.drop(['target'], axis=1)
test2=test1.drop(['target'], axis=1)

mean_train2=train2.mean()
std_train2=train2.std()

train2=(train2-mean_train2)/std_train2
test2=(test2-mean_train2)/std_train2

dic_to_One_Hot = {0 : [1,0], 1 : [0,1]}

X_train=np.array(train2)
X_test=np.array(test2)

y_train_cls=np.array(train1['target'])
y_train=np.array(list(train1['target'].map(dic_to_One_Hot)))

y_test_cls=test1['target']
y_test=np.array(list(y_test_cls.map(dic_to_One_Hot)))
y_test_cls=np.array(y_test_cls)



## ML Tuning

In [5]:
cols = train2.columns.values  #On a besoin des colonnes pour plus tard 
                                # On observe le grand X et pas le petit x ...

ntrain = train2.shape[0]

ntest =test2.shape[0]

seed_fix = 5                   # Pour reproduire les résultats

In [6]:
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        return self.clf.fit(x,y).feature_importances_

In [7]:
NFOLDS = 5 
kf = KFold(ntrain, n_folds= NFOLDS, random_state=seed_fix)



def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [8]:

rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

In [9]:
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

In [10]:
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

In [11]:
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

In [12]:
rf = SklearnHelper(clf=RandomForestClassifier, seed=seed_fix , params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=seed_fix, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=seed_fix , params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=seed_fix, params=gb_params)

## Learning zone

In [13]:
et_oof_train, et_oof_test = get_oof(et, X_train, y_train_cls, X_test) # Extra Trees
rf_oof_train, rf_oof_test = get_oof(rf,X_train, y_train_cls, X_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, X_train, y_train_cls, X_test) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,X_train, y_train_cls, X_test) # Gradient Boost

print("I am done learning :D")

C:\Users\Delanoue\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:303: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.



I am done learning :D


In [14]:
rf_feature = rf.feature_importances(X_train,y_train_cls)
et_feature = et.feature_importances(X_train, y_train_cls)
ada_feature = ada.feature_importances(X_train, y_train_cls)
gb_feature = gb.feature_importances(X_train,y_train_cls)

#On met plutot dans un dataFrame
feature_dataframe = pd.DataFrame( {'features': cols,
     'Random Forest Importance des features': rf_feature,
     'Extra Trees Importance des features': et_feature,
      'AdaBoost Importance des features': ada_feature,
    'Gradient Boost Importance des features': gb_feature
    })

C:\Users\Delanoue\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:303: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.



In [15]:
rf_pred=rf.predict(X_test)
et_pred=et.predict(X_test)
ada_pred=ada.predict(X_test)
gb_pred=gb.predict(X_test)

## Classification Reports

In [16]:
from sklearn import metrics
print(metrics.classification_report(rf_pred, y_test_cls))

             precision    recall  f1-score   support

          0       0.68      0.59      0.63        82
          1       0.35      0.44      0.39        41

avg / total       0.57      0.54      0.55       123



In [17]:
print(metrics.classification_report(et_pred, y_test_cls))

             precision    recall  f1-score   support

          0       0.82      0.63      0.71        92
          1       0.35      0.58      0.43        31

avg / total       0.70      0.62      0.64       123



In [18]:
print(metrics.classification_report(ada_pred, y_test_cls))

             precision    recall  f1-score   support

          0       0.70      0.64      0.67        78
          1       0.46      0.53      0.49        45

avg / total       0.62      0.60      0.61       123



In [19]:
print(metrics.classification_report(gb_pred, y_test_cls))

             precision    recall  f1-score   support

          0       0.68      0.65      0.66        74
          1       0.50      0.53      0.51        49

avg / total       0.61      0.60      0.60       123



## Feature Importance Visualisation 

In [20]:
trace = go.Scatter(x = feature_dataframe['features'].values,
    y = feature_dataframe['Random Forest Importance des features'].values,
    
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Random Forest Importance des features'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)

data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Importance des features',
    hovermode= 'closest',
    yaxis=dict(
        title= 'Importance des features',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig,filename='scatter2010')

In [21]:
trace = go.Scatter(
    y = feature_dataframe['Extra Trees Importance des features'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
        color = feature_dataframe['Extra Trees Importance des features'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Extra Trees Importance des features',
    hovermode= 'closest',

    yaxis=dict(
        title= 'Importance des features',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [22]:
trace = go.Scatter(
    y = feature_dataframe['AdaBoost Importance des features'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
        color = feature_dataframe['AdaBoost Importance des features'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'AdaBoost Importance des features',
    hovermode= 'closest',
    yaxis=dict(
        title= 'Importance des features',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [23]:
trace = go.Scatter(
    y = feature_dataframe['Gradient Boost Importance des features'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
        color = feature_dataframe['Gradient Boost Importance des features'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Gradient Boosting Importance des features',
    hovermode= 'closest',
    yaxis=dict(
        title= 'Importance des features',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')